###Group 14:

1. Ravi Shankar -->		ravishankar2k4@gmail.com









In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 =nn.Sequential(nn.Conv2d(1,16, 3, padding=0),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout(0.1)) #input -28x28 OUtput- 26x26
        self.conv2 =nn.Sequential(nn.Conv2d(16,16, 3, padding=0),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout(0.1)) #input -26x26 OUtput-24x24 
        self.conv3 =nn.Sequential(nn.Conv2d(16,16, 3, padding=0),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout(0.1)) #input -24x24 OUtput- 22x22

        #Max Pooling
        self.pool1 = nn.MaxPool2d(2, 2) #output 11x11x16

        # Transition layer
        self.conv4 =nn.Sequential(nn.Conv2d(16,8,1,padding=0)) #output 11x11x8

        #Convolution block 2
        self.conv5=nn.Sequential(nn.Conv2d(8, 16, 3, padding=0),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout(0.1)) #input -11x11 OUtput-  9x9
        self.conv6 =nn.Sequential(nn.Conv2d(16, 16, 3, padding=0),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout(0.1)) #input -9x9 OUtput- 7x7
        self.conv7 =nn.Sequential(nn.Conv2d(16, 16, 3, padding=0),nn.ReLU(),nn.BatchNorm2d(16),nn.Dropout(0.1)) #input -7x7 OUtput- 5x5

        #GAP
        self.gap = nn.Sequential(nn.AvgPool2d(kernel_size=5)) # output_size = 1x1x16
        
        # Transition layer
        self.conv8 =nn.Sequential(nn.Conv2d(16,10,1,padding=0)) #output 1x1x10


    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool1(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.gap(x)        
        x = self.conv8(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,320
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 16, 22, 22]           2,320
             ReLU-10           [-1, 16, 22, 22]               0
      BatchNorm2d-11           [-1, 16, 22, 22]              32
          Dropout-12           [-1, 16, 22, 22]               0
        MaxPool2d-13           [-1, 16, 11, 11]               0
           Conv2d-14            [-1, 8,

In [4]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

for epoch in range(1, 21):
    print("Epoch:",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch: 1


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0445, Accuracy: 9869/10000 (98.69%)

Epoch: 2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0176, Accuracy: 9943/10000 (99.43%)

Epoch: 19


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0169, Accuracy: 9944/10000 (99.44%)

Epoch: 20


loss=0.055301252752542496 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 57.29it/s]



Test set: Average loss: 0.0194, Accuracy: 9936/10000 (99.36%)



##Highest accuracy of 99.45% -Epoch 15
## Target accuracy of 99.40% achieved in Epoch 8
